In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,48001
2,Huelva,Confirmados PDIA 14 días,1011
3,Huelva,Tasa PDIA 14 días,"197,01073718261006"
4,Huelva,Confirmados PDIA 7 días,496
5,Huelva,Tasa PDIA 7 dias,"96,65413021026171"
6,Huelva,Total Confirmados,48207
7,Huelva,Curados,45913
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  48001.0


/tmp/ipykernel_22987/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13345.0


/tmp/ipykernel_22987/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_22987/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_22987/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_22987/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 804 personas en los últimos 7 días 

Un positivo PCR cada 398 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,48001.0,496.0,1011.0,513170.0,96.654130,197.010737,96.0
Huelva-Costa,28359.0,275.0,574.0,289548.0,94.975617,198.240015,51.0
Condado-Campiña,14984.0,170.0,345.0,156231.0,108.813232,220.826853,34.0
Huelva (capital),13345.0,179.0,361.0,143837.0,124.446422,250.978538,30.0
Cartaya,2283.0,30.0,70.0,20083.0,149.380073,348.553503,12.0
Sierra de Huelva-Andévalo Central,4230.0,51.0,92.0,67391.0,75.677761,136.516746,11.0
Palos de la Frontera,1195.0,18.0,47.0,11742.0,153.295861,400.272526,8.0
Palma del Condado (La),1492.0,17.0,33.0,10801.0,157.392834,305.527266,6.0
Valverde del Camino,901.0,20.0,36.0,12750.0,156.862745,282.352941,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),17.0,2.0,10.0,243.0,823.045267,4115.226337,0.0
Paterna del Campo,275.0,27.0,55.0,3457.0,781.024009,1590.974834,5.0
Puerto Moral,8.0,3.0,3.0,278.0,1079.136691,1079.136691,NaN
Corteconcepción,49.0,2.0,3.0,536.0,373.134328,559.701493,0.0
Palos de la Frontera,1195.0,18.0,47.0,11742.0,153.295861,400.272526,8.0
Bollullos Par del Condado,1392.0,35.0,56.0,14387.0,243.275179,389.240286,5.0
Manzanilla,152.0,5.0,8.0,2118.0,236.071766,377.714825,2.0
Cartaya,2283.0,30.0,70.0,20083.0,149.380073,348.553503,12.0
Alosno,249.0,1.0,13.0,3933.0,25.425884,330.536486,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,249.0,1.0,13.0,3933.0,25.425884,330.536486,0.0,0.076923
Granada de Río-Tinto (La),17.0,2.0,10.0,243.0,823.045267,4115.226337,0.0,0.200000
Ayamonte,2130.0,4.0,14.0,21104.0,18.953753,66.338135,1.0,0.285714
Lepe,3067.0,3.0,10.0,27880.0,10.760402,35.868006,1.0,0.300000
Bonares,476.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Rociana del Condado,693.0,6.0,17.0,7939.0,75.576269,214.132762,0.0,0.352941
Moguer,2136.0,21.0,57.0,21867.0,96.035121,260.666758,4.0,0.368421
Aljaraque,1714.0,19.0,51.0,21474.0,88.479091,237.496507,2.0,0.372549
Palos de la Frontera,1195.0,18.0,47.0,11742.0,153.295861,400.272526,8.0,0.382979
